# 💧 OpenAI API Tutorial for Alabama Water Institute (AWI)
Welcome to this beginner-friendly tutorial on using the OpenAI API in Python.

In this tutorial, you'll learn how to:
- Generate text using GPT-4
- Use function calling (structured output)
- Generate images with DALL·E
- Handle errors safely

In [ ]:
# ✅ Install OpenAI SDK (skip if already installed)
!pip install openai

## 🔐 Set Up Your API Key
I recommend saving your key in an environment variable for security.

In [ ]:
import os
import openai
from google.colab import userdata

# Set your OpenAI API key
openai.api_key = userdata.get('OPENAI_API_KEY')  # Make sure you've set this on Colab or locally

## 🧠 Generate Text with ChatGPT

In [ ]:
from openai import OpenAI

# Initialize client
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# Call Chat API
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant for water research."},
        {"role": "user", "content": "Explain the hydrological cycle in simple terms."}
    ]
)

# Print the response
print(response.choices[0].message.content)

Sure, the hydrological cycle, also known as the water cycle, is a continuous process in which water changes states and locations around the planet. 

Here it is in simple terms:

1. Evaporation: Water from the oceans, lakes, rivers, and the ground surface turns into water vapor due to the Sun's heat. Plants also contribute to this through a process called transpiration.

2. Condensation: The water vapor rises into the air, where it cools down and turns back into a liquid, forming clouds. This is condensation.

3. Precipitation: When the cloud particles become too heavy to remain suspended in the cloud, they fall back to the Earth as rain, snow, hail, or sleet. This process is called precipitation.

4. Runoff and Infiltration: The water that falls to the Earth either evaporates, becomes part of the ground water (infiltration), or runs off into rivers and streams, which eventually lead back to the oceans and the cycle begins again.

Remember that this cycle is nature's way of recycling w

## 📷 Generate Images with DALL·E

In [28]:
import requests

# Initialize client
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY')) # Use the client object and get API key from userdata

# Generate image with a text prompt using DALL·E model
response = client.images.generate( # Use client.images.generate
    model="dall-e-2",  # DALL·E 2 model
    prompt="a futuristic water research facility in Alabama",  # Text prompt
    n=1,  # Number of images to generate
    size="1024x1024",  # Image size, options: "256x256", "512x512", "1024x1024"
)

# Extract the image URL from the response
image_url = response.data[0].url # Access data and url using dot notation
print("Generated Image URL:", image_url)

Generated Image URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-RsZ0ZCzHPOcYWn1fyTNGCev0/user-AnumEixfTKOOCtRJKTwfv7Ib/img-uJcpwZ2zZUMjGczlXs23xsMG.png?st=2025-05-27T18%3A28%3A48Z&se=2025-05-27T20%3A28%3A48Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=cc612491-d948-4d2e-9821-2683df3719f5&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-05-27T17%3A06%3A51Z&ske=2025-05-28T17%3A06%3A51Z&sks=b&skv=2024-08-04&sig=7bAn9zHVtP1Sj5j7Q2T3Gb/zS7y4UZSS4p/LAwL%2BRYk%3D


## 🔍Analyze text

In [ ]:
# Sample reviews collection
reviews = [
    "The product was amazing, I absolutely love it!",
    "It’s okay, but it could be better. Not bad, not great.",
    "Terrible experience. The product broke after one day.",
    "Best purchase ever! Worth every penny.",
    "The quality was poor, very disappointing."
]

# Function to analyze sentiment using ChatGPT
def analyze_sentiment(reviews):
    sentiments = []
    for review in reviews:
        response = client.chat.completions.create( # Use client.chat.completions.create
            model="gpt-4",  # Use GPT-4 for the most accurate results
            messages=[ # Use the messages parameter for chat models
                {"role": "system", "content": "You are a helpful assistant for sentiment analysis. Respond only with 'Positive', 'Neutral', or 'Negative'."}, # Add a system message to guide the model
                {"role": "user", "content": f"Analyze the sentiment of the following review: '{review}'."} # Format the user message
            ],
            max_tokens=10,  # Only need the sentiment, so a short response
            temperature=0.0,  # Set temperature to 0 for a more deterministic answer
        )
        sentiment = response.choices[0].message.content.strip() # Access content using .message.content
        sentiments.append(sentiment)
    return sentiments

# Get sentiment for each review
sentiments = analyze_sentiment(reviews)

# Print out the results
for review, sentiment in zip(reviews, sentiments):
    print(f"Review: {review}\nSentiment: {sentiment}\n")

Review: The product was amazing, I absolutely love it!
Sentiment: Positive

Review: It’s okay, but it could be better. Not bad, not great.
Sentiment: Neutral

Review: Terrible experience. The product broke after one day.
Sentiment: Negative

Review: Best purchase ever! Worth every penny.
Sentiment: Positive

Review: The quality was poor, very disappointing.
Sentiment: Negative



## 🚧 Handle Errors Safely

When working with external APIs like OpenAI, it's crucial to handle potential errors gracefully. Common errors include network issues, invalid API keys, rate limits, and model-specific errors. You can use `try...except` blocks to catch these errors and provide informative feedback or retry mechanisms.

In [ ]:
from openai import OpenAI, APIError, RateLimitError
from google.colab import userdata
import time

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def safe_chat_completion(prompt, model="gpt-4", max_tokens=100, temperature=0.7, retries=3):
    """
    Safely call the chat completion API with retry logic.
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            return response.choices[0].message.content

        except RateLimitError:
            wait_time = (2 ** attempt) # Exponential backoff
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except APIError as e:
            print(f"API Error: {e}")
            break # Don't retry for other API errors
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break # Don't retry for unexpected errors

    return "Failed to get a response after multiple retries."

# Example usage with a potentially problematic prompt (e.g., very long or complex)
# You can replace this with a prompt that might cause an error to test the handling
test_prompt = "Explain a complex scientific concept in simple terms, making it very long and detailed."
response = safe_chat_completion(test_prompt)
print("\nSafe API Call Response:")
print(response)


Safe API Call Response:
Alright, let's take the concept of Quantum Mechanics, which is one of the pillars of modern physics, and simplify it as much as possible. This is going to be a long ride so buckle up!

Quantum Mechanics, at its core, is a physical science dealing with the behavior of matter and light at the molecular, atomic, nuclear, and even smaller, microscopic levels. 

1. **Wave-Particle Duality**: To start with, one of the most mind-bending theories in quantum


# 🔥 PyTorch For Neural Networks Tutorial for the Alabama Water Insitute (AWI)

##1. Introduction


Overview of PyTorch
PyTorch is an open-source deep learning library for applications like computer vision and NLP. It’s known for its flexibility, ease of use, and support for GPU acceleration.


Why PyTorch for Neural Networks?
Dynamic Computation Graph: Flexible network design during training.


GPU Support: Speeds up training with hardware acceleration.


Neural Networks in Water Research
Neural networks are used for:


Predicting Water Quality: Forecasting future water conditions.


Modeling Environmental Changes: Analyzing trends in water data.


Optimizing Water Distribution: Forecasting demand to improve efficiency.


Goal of the Tutorial
This tutorial covers building, training, and evaluating a neural network in PyTorch, with potential applications to water-related problems.



##2. Set Up and Dependencies

- Install Libraries:

In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

- Import Necessary Libraries:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, TensorDataset

##3. Data Preparation

- Dataset Example: If you have water-related datasets, use that. If not, you could use a sample dataset, like the Iris dataset.

- Preprocessing:

 - Loading data into PyTorch tensors.

 - Normalizing data if necessary.

- Example code:

In [ ]:
# Example dataset
# Replace with actual data
data = torch.randn(100, 10)  # Example: 100 samples, 10 features
targets = torch.randn(100, 1)  # Example: 100 samples, 1 target

input_size = data.shape[1]
output_size = targets.shape[1]

dataset = TensorDataset(data, targets)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Input size: {input_size}")
print(f"Output size: {output_size}")

Input size: 10
Output size: 1


##4. Building a Simple Neural Network Model

- Define the Model: Use `torch.nn.Module` to define a simple fully connected neural network.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Dynamically set input features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_size)  # Dynamically set output features

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

##5. Training the Model

- Define the Loss Function and Optimizer:

In [ ]:
model = NeuralNetwork(input_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error for regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

- Training Loop:

In [ ]:
epochs = 10
for epoch in range(epochs):
    for batch_data, batch_targets in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [1/500], Loss: 0.0000
Epoch [2/500], Loss: 0.0009
Epoch [3/500], Loss: 0.0006
Epoch [4/500], Loss: 0.0000
Epoch [5/500], Loss: 0.0009
Epoch [6/500], Loss: 0.0015
Epoch [7/500], Loss: 0.0002
Epoch [8/500], Loss: 0.0010
Epoch [9/500], Loss: 0.0018
Epoch [10/500], Loss: 0.0000
Epoch [11/500], Loss: 0.0001
Epoch [12/500], Loss: 0.0008
Epoch [13/500], Loss: 0.0006
Epoch [14/500], Loss: 0.0002
Epoch [15/500], Loss: 0.0010
Epoch [16/500], Loss: 0.0004
Epoch [17/500], Loss: 0.0016
Epoch [18/500], Loss: 0.0001
Epoch [19/500], Loss: 0.0002
Epoch [20/500], Loss: 0.0010
Epoch [21/500], Loss: 0.0002
Epoch [22/500], Loss: 0.0001
Epoch [23/500], Loss: 0.0003
Epoch [24/500], Loss: 0.0007
Epoch [25/500], Loss: 0.0002
Epoch [26/500], Loss: 0.0003
Epoch [27/500], Loss: 0.0015
Epoch [28/500], Loss: 0.0019
Epoch [29/500], Loss: 0.0003
Epoch [30/500], Loss: 0.0006
Epoch [31/500], Loss: 0.0005
Epoch [32/500], Loss: 0.0002
Epoch [33/500], Loss: 0.0007
Epoch [34/500], Loss: 0.0002
Epoch [35/500], Loss: 0

##6. Evaluating the Model

- Test the Model on New Data: Evaluate the performance of the model after training.



##7. Integrating OpenAI API (for advanced usage)

Since you have already been taught about the OpenAI API, you could integrate it with PyTorch for applications like using pre-trained models or generating text based on water-related predictions.

Example integration snippet:

In [ ]:
import openai
from google.colab import userdata
import tiktoken
import random
import torch

def generate_prediction(input_data):
    client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

    # Get the encoding for the model
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-instruct")
    max_tokens = 4097  # Maximum context length for gpt-3.5-turbo-instruct
    completion_tokens = 50 # Tokens reserved for the completion
    available_tokens = max_tokens - completion_tokens

    # Estimate token count per data point
    if input_data.shape[0] > 0:
        # Take the string representation of the first data point
        sample_data_string = str(input_data[0])
        tokens_per_data_point = len(encoding.encode(sample_data_string))
        # Add some buffer for formatting (commas, brackets, etc.)
        estimated_tokens_per_data_point = tokens_per_data_point + 5
    else:
        return "Input data is empty."

    # Calculate how many data points can fit
    # Reserve some tokens for the prompt text itself
    prompt_text_tokens = len(encoding.encode("Given the data , predict the water quality levels."))
    data_tokens_capacity = available_tokens - prompt_text_tokens

    if estimated_tokens_per_data_point > 0:
        num_data_points_to_include = data_tokens_capacity // estimated_tokens_per_data_point
    else:
        num_data_points_to_include = 0


    # Select data points to include
    if num_data_points_to_include > 0 and input_data.shape[0] > num_data_points_to_include:
        # Randomly sample the calculated number of data points
        indices_to_include = random.sample(range(input_data.shape[0]), min(input_data.shape[0], num_data_points_to_include))
        selected_data = input_data[indices_to_include]
        print(f"Reducing data size to {min(input_data.shape[0], num_data_points_to_include)} data points.")
    else:
        selected_data = input_data # Include all data if it fits or is empty
        if input_data.shape[0] > 0:
            print("Including all data points.")
        else:
            print("No data points to include.")


    data_string = str(selected_data)
    prompt=f"Given the data {data_string}, predict the water quality levels."


    try:
        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=completion_tokens
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"An error occurred: {e}"


# Example usage with the data tensor from the previous cell
print(generate_prediction(data))

Reducing data size to 56 data points.
An error occurred: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 5453 tokens (5403 in your prompt; 50 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


#🛠️ Creating a pipeline

##Conceptualizing the Pipeline


1. Problem Understanding
To begin, let’s first define a scenario where we could use NNs, RAG, and LLMs together. Consider a question-answering (QA) system that needs to:

 - Retrieve relevant documents from a knowledge base (RAG component).

 - Process the text and use a neural network (NN) to better understand it (NN component).

 - Generate a detailed response with a language model, enriched by the retrieved documents (LLM component).

2. Components of the Pipeline
Neural Network (NN): A neural network could be used for text preprocessing, such as extracting features or understanding the semantics of the input query or text.

 - Retrieval-Augmented Generation (RAG): This module retrieves relevant documents from an external knowledge base (e.g., a database of articles or papers) based on the query.

 - Large Language Model (LLM): The LLM generates a response based on both the query and the retrieved documents, leveraging its pretrained knowledge to produce detailed, context-rich answers.

## Modeling the Pipeline

Step 1: Define Your Task
For this tutorial, let’s assume we are building a QA system that retrieves articles from a document database (using RAG) and generates responses to questions using a large language model (like GPT).

We’ll define the pipeline in three stages:

 1. Document Retrieval: Fetch documents from a knowledge base.

 2. Document Understanding: Process the documents using a neural network to extract features and understand them.

 3. Answer Generation: Use an LLM to generate a final answer, augmented by the retrieved documents.


Step 2: Design the Pipeline Workflow
Input: User query or question.

- Stage 1 (RAG - Retrieval):

 - Use a retrieval system (such as Elasticsearch or FAISS) to find relevant documents.

- Stage 2 (NN - Document Understanding):

 - Preprocess the retrieved documents (e.g., sentence segmentation, embedding generation) with a neural network.

- Stage 3 (LLM - Answer Generation):

 - Use the LLM to generate an answer by providing the user query and the processed documents as context.


Part 3: Implementing the Pipeline

- Step 1: Setup Dependencies
You’ll need a few libraries for this task:

 - Hugging Face's Transformers for the LLM (like GPT-3, GPT-4, or BERT for document understanding).

 - FAISS or Elasticsearch for the retrieval mechanism.

 - PyTorch or TensorFlow for building the NN components.

 Install these dependencies first:

In [ ]:
!pip install transformers faiss-cpu torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.0 MB/s eta 0:00:00


- Step 2: Define your data
 - Example document: [Water_and_Flooding_Services](https://docs.google.com/document/d/1IX2bKinZLEZP_nK1jdAGLShDG8X1Chye/edit?usp=sharing&ouid=118264855319059337623&rtpof=true&sd=true)
 - Convert data to json/make it machine readable:

In [13]:
documents=[
  {
    "title": "Water and Flooding Services",
    "content": "Flooding is one of the most common and destructive natural disasters, impacting communities worldwide. Understanding and addressing flooding is essential for safeguarding lives, infrastructure, and the environment. Various services and technologies are designed to monitor, predict, and manage floods effectively. These services can range from monitoring flood risks to responding to flood events."
  },
  {
    "title": "Key Aspects of Water and Flooding Services",
    "sections": [
      {
        "heading": "1. Flood Prediction and Forecasting",
        "content": [
          "Hydrological Models: These models simulate the flow of water in rivers, streams, and catchments to predict the likelihood of floods. They rely on weather data, historical rainfall, and water level measurements.",
          "Weather Forecasting: Meteorological agencies use sophisticated models to forecast rainfall, which is a major trigger for flooding. Forecasting can provide early warnings about impending storms and floods."
        ]
      },
      {
        "heading": "2. Flood Monitoring",
        "content": [
          "River Gauging Stations: These stations monitor water levels in rivers and streams. Changes in water levels can indicate potential flooding.",
          "Satellite Monitoring: Remote sensing technologies, including satellites, can provide real-time data on water levels, rainfall, and snowmelt, allowing for better monitoring of large areas."
        ]
      },
      {
        "heading": "3. Flood Risk Assessment",
        "content": [
          "Flood Mapping: Flood maps are created based on historical data and simulations to determine areas that are at risk of flooding. These maps help in urban planning and building regulations, ensuring that structures are built in safe locations.",
          "Risk Models: Flood risk models integrate data on topography, hydrology, and infrastructure to assess flood risks in different regions."
        ]
      },
      {
        "heading": "4. Flood Mitigation and Management",
        "content": [
          "Flood Defenses: Infrastructure such as levees, dams, reservoirs, and flood barriers are constructed to protect vulnerable areas from floods.",
          "Natural Solutions: Restoring wetlands, forests, and other natural landscapes can help absorb excess water and reduce the impact of flooding.",
          "Floodplain Zoning: Floodplain zoning regulations are used to restrict construction in high-risk areas and encourage flood-resistant designs in at-risk locations."
        ]
      },
      {
        "heading": "5. Flood Response and Recovery",
        "content": [
          "Flood Warning Systems: Automated flood warning systems alert communities about potential floods through sirens, text messages, or apps. Early warnings help people evacuate and protect assets in time.",
          "Emergency Services: In case of flooding, emergency services, such as fire and rescue teams, are deployed to help evacuate people, provide medical assistance, and rescue stranded individuals.",
          "Post-Flood Recovery: Once a flood event subsides, recovery services help repair damaged infrastructure, provide financial support, and assist in long-term rebuilding efforts."
        ]
      }
    ]
  },
  {
    "title": "Technology in Flood and Water Management",
    "sections": [
      {
        "heading": "1. Artificial Intelligence (AI)",
        "content": "AI can be used to enhance flood prediction models by analyzing historical data, weather patterns, and real-time environmental data to forecast floods more accurately."
      },
      {
        "heading": "2. Internet of Things (IoT)",
        "content": "IoT sensors can be deployed in rivers, dams, and other flood-prone areas to collect real-time data on water levels, rainfall, and environmental conditions. This data can be analyzed for early flood detection."
      },
      {
        "heading": "3. Geospatial Technologies",
        "content": "Geographic Information System (GIS) and remote sensing technologies enable the creation of detailed flood maps, providing insights into flood risks and potential impacts on communities."
      }
    ]
  },
  {
    "title": "Applications of Water and Flooding Services",
    "sections": [
      {
        "heading": "1. Disaster Risk Reduction",
        "content": "By predicting floods and providing early warnings, communities can reduce the damage caused by floods and take preventive measures."
      },
      {
        "heading": "2. Urban Planning",
        "content": "Flood risk assessments and floodplain zoning play an essential role in the design of cities and towns, ensuring buildings and infrastructure are placed in safe areas."
      },
      {
        "heading": "3. Agriculture and Water Resources",
        "content": "Flood management is crucial for agriculture, as floods can destroy crops and disrupt water resources. Early warnings can help farmers protect their crops and manage irrigation systems more effectively."
      },
      {
        "heading": "4. Insurance",
        "content": "Flood risk mapping helps insurance companies assess the risk of floods in different regions, allowing them to set premiums based on the likelihood of flood damage."
      }
    ]
  },
  {
    "title": "Flooding and Water Services for Communities",
    "sections": [
      {
        "heading": "1. Flood Insurance",
        "content": "Coverage that helps homeowners and businesses recover financially after a flood. Flood insurance policies are designed to cover damages not typically included in regular homeowners insurance."
      },
      {
        "heading": "2. Community Flood Preparedness Programs",
        "content": "Many governments offer programs to educate communities about flood risks, evacuation plans, and emergency preparedness."
      },
      {
        "heading": "3. Flood Response Services",
        "content": "Local authorities often have dedicated flood response teams that coordinate evacuation plans, set up temporary shelters, and provide resources to affected individuals during and after flooding events."
      }
    ]
  },
  {
    "title": "Conclusion",
    "content": "Flooding is a complex challenge that requires a combination of predictive modeling, infrastructure planning, real-time monitoring, and community preparedness. By leveraging modern technologies, such as AI, IoT, and satellite data, communities can improve their flood resilience and minimize the impact of floods on their populations. Effective water and flooding services are crucial for managing and mitigating the risks associated with floods, ensuring both human safety and environmental protection."
  }
]


- Step 3: Implement the RAG Component (Document Retrieval)
For the retrieval step, we'll use FAISS (Facebook AI Similarity Search) for document retrieval.

In [14]:
import faiss
import numpy as np

# Example: FAISS index setup for document retrieval
dimension = 768  # Size of the embedding (depends on your model, e.g., BERT)
index = faiss.IndexFlatL2(dimension)  # L2 distance index for FAISS

# Example documents (replace with actual embeddings of your documents)
document_embeddings = np.random.rand(len(documents), dimension).astype('float32')

# Add document embeddings to FAISS index
index.add(document_embeddings)

def retrieve_documents(query_embedding, top_k=3):
    # Search for the top_k most similar documents
    distances, indices = index.search(query_embedding, top_k)
    return [documents[i] for i in indices[0]]

# Example: Query embedding (replace with actual model embeddings)
query_embedding = np.random.rand(1, dimension).astype('float32')

# Retrieve top 3 similar documents
retrieved_docs = retrieve_documents(query_embedding)
print(retrieved_docs)

[{'title': 'Conclusion', 'content': 'Flooding is a complex challenge that requires a combination of predictive modeling, infrastructure planning, real-time monitoring, and community preparedness. By leveraging modern technologies, such as AI, IoT, and satellite data, communities can improve their flood resilience and minimize the impact of floods on their populations. Effective water and flooding services are crucial for managing and mitigating the risks associated with floods, ensuring both human safety and environmental protection.'}, {'title': 'Technology in Flood and Water Management', 'sections': [{'heading': '1. Artificial Intelligence (AI)', 'content': 'AI can be used to enhance flood prediction models by analyzing historical data, weather patterns, and real-time environmental data to forecast floods more accurately.'}, {'heading': '2. Internet of Things (IoT)', 'content': 'IoT sensors can be deployed in rivers, dams, and other flood-prone areas to collect real-time data on wate

Here, FAISS helps to quickly search and retrieve the most relevant documents based on a query embedding.

- Step 4: Implement the NN Component (Document Understanding)
Now, let's process the retrieved documents using a neural network model. For this, we'll use BERT to embed the documents and query.

In [16]:
from transformers import BertTokenizer, BertModel
import torch

# Initialize BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def encode_document(doc):
    # Extract text content from the document dictionary
    if "content" in doc:
        text = doc["content"]
    elif "sections" in doc:
        # Concatenate content from all sections if available
        text = " ".join([section["content"] for section in doc["sections"] if isinstance(section["content"], str)])
    else:
        text = "" # Handle cases with no content or sections

    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = bert_model(**inputs)
    # Use mean pooling for sentence embedding, handle potential empty outputs
    if outputs.last_hidden_state.shape[1] > 0:
        return outputs.last_hidden_state.mean(dim=1).detach().numpy()
    else:
        return np.zeros((1, bert_model.config.hidden_size), dtype=np.float32)


# Example: Encoding the retrieved documents
encoded_docs = [encode_document(doc) for doc in retrieved_docs]

In this step, we use BERT to encode the documents into embeddings. You can use any other pre-trained model based on your needs.

- Step 5: Implement the LLM Component (Answer Generation)
Finally, let’s use an LLM like GPT-3 or GPT-4 to generate an answer based on the query and the retrieved documents.

In [24]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Initialize GPT model and tokenizer (or use GPT-3 via OpenAI API)
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set pad token ID if it's not already set
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token


def generate_answer(query, retrieved_docs):
    # Extract text content from each document and join them
    document_texts = []
    for doc in retrieved_docs:
        if "content" in doc:
            document_texts.append(doc["content"])
        elif "sections" in doc:
            # Concatenate content from all sections if available
            section_content = " ".join([section["content"] for section in doc["sections"] if isinstance(section["content"], str)])
            document_texts.append(section_content)

    context = " ".join(document_texts) + " " + query  # Combine query and retrieved docs

    # Encode the context and return attention mask
    inputs = gpt_tokenizer.encode_plus(
        context,
        return_tensors='pt',
        padding=True,  # Enable padding
        truncation=True # Enable truncation if context is too long
    )
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    outputs = gpt_model.generate(
        input_ids,
        attention_mask=attention_mask, # Pass attention mask
        max_new_tokens=200, # Use max_new_tokens instead of max_length
        num_return_sequences=1,
        pad_token_id=gpt_tokenizer.pad_token_id # Pass pad token id
    )
    return gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example: Generate answer based on the query and retrieved documents
query = "What is the impact of heavy rainfall on flood risk?"
answer = generate_answer(query, retrieved_docs)
print(answer)

Flooding is a complex challenge that requires a combination of predictive modeling, infrastructure planning, real-time monitoring, and community preparedness. By leveraging modern technologies, such as AI, IoT, and satellite data, communities can improve their flood resilience and minimize the impact of floods on their populations. Effective water and flooding services are crucial for managing and mitigating the risks associated with floods, ensuring both human safety and environmental protection. AI can be used to enhance flood prediction models by analyzing historical data, weather patterns, and real-time environmental data to forecast floods more accurately. IoT sensors can be deployed in rivers, dams, and other flood-prone areas to collect real-time data on water levels, rainfall, and environmental conditions. This data can be analyzed for early flood detection. Geographic Information System (GIS) and remote sensing technologies enable the creation of detailed flood maps, providing

Here, GPT-2 (or GPT-3 via API) generates the final answer, using both the query and the retrieved documents as context.

Part 4: Putting It All Together
Now, let’s combine all components into a single pipeline:

In [25]:
def qa_pipeline(query):
    # Step 1: Retrieve documents (RAG)
    # Encode the query using the BERT model and tokenizer
    query_inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
    query_outputs = bert_model(**query_inputs)
    query_embedding = query_outputs.last_hidden_state.mean(dim=1).detach().numpy()

    retrieved_docs = retrieve_documents(query_embedding)

    # Step 2: Generate answer (LLM)
    answer = generate_answer(query, retrieved_docs)
    return answer

# Example: Running the pipeline
query = "What measures can prevent flooding in urban areas?"
final_answer = qa_pipeline(query)
print(final_answer)

AI can be used to enhance flood prediction models by analyzing historical data, weather patterns, and real-time environmental data to forecast floods more accurately. IoT sensors can be deployed in rivers, dams, and other flood-prone areas to collect real-time data on water levels, rainfall, and environmental conditions. This data can be analyzed for early flood detection. Geographic Information System (GIS) and remote sensing technologies enable the creation of detailed flood maps, providing insights into flood risks and potential impacts on communities. By predicting floods and providing early warnings, communities can reduce the damage caused by floods and take preventive measures. Flood risk assessments and floodplain zoning play an essential role in the design of cities and towns, ensuring buildings and infrastructure are placed in safe areas. Flood management is crucial for agriculture, as floods can destroy crops and disrupt water resources. Early warnings can help farmers prote